In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_5896/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
import os
print(os.getcwd())


/notebooks


In [4]:
from pathlib import WindowsPath

In [5]:
path_training = WindowsPath("C:/Users/Salva/Documents/AI Engeneering Ironhack/Week 4/lab-natural-language-processing/data/kg_train.csv")



## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(path_training,encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

NotImplementedError: cannot instantiate 'WindowsPath' on your system

### Let's divide the training and test set into two partitions

In [6]:
from sklearn.model_selection import train_test_split

# Assuming `data` contains the reduced training set (1000 rows)
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Check the shapes to confirm the split
print("Training set shape:", train_data.shape)
print("Validation set shape:", val_data.shape)
train_data.head

Training set shape: (800, 2)
Validation set shape: (200, 2)


<bound method NDFrame.head of                                                   text  label
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1
535  I have not been able to reach oscar this am. W...      0
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0
557  I can have it announced here on Monday - can't...      0
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1
..                                                 ...    ...
106  7653 2612ADAMA IBRAHIM________________________...      1
270             What does that mean for our schedules?      0
860  Dear Friend,My Compliment to you,I guess this ...      1
435  Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...      1
102  Let me know if today or tomorrow works for you...      0

[800 rows x 2 columns]>

## Data Preprocessing

In [4]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [5]:
from bs4 import BeautifulSoup, Comment
import pandas as pd

# Function to clean HTML content
def clean_html(text):
    if pd.isna(text):  # Handle missing values (NaN)
        return ""
    
    soup = BeautifulSoup(text, 'html.parser')

    # Remove script and style elements
    for script in soup(["script", "style"]):
        script.decompose()

    # Remove HTML comments
    comments = soup.findAll(text=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()

    # Extract text and return cleaned content
    return soup.get_text(separator=" ")

# Apply BeautifulSoup to the entire dataset
def clean_dataset(dataset, text_column):
    dataset[text_column] = dataset[text_column].apply(clean_html)
    return dataset


In [9]:
# Use the column name 'text' instead of a numerical index
text_column = 'text'  # Column label for the text data

# Clean the training and validation sets
train_data_cleaned = clean_dataset(train_data, text_column)
val_data_cleaned = clean_dataset(val_data, text_column)

# Display the cleaned data to check
print(train_data_cleaned.head())
print(val_data_cleaned.head())


C:\Users\Salva\AppData\Local\Temp\ipykernel_1504\1710886297.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  comments = soup.findAll(text=lambda text: isinstance(text, Comment))
C:\Users\Salva\AppData\Local\Temp\ipykernel_1504\1710886297.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


                                                  text  label
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1
535  I have not been able to reach oscar this am. W...      0
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0
557  I can have it announced here on Monday - can't...      0
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1
                                                  text  label
521  Dear Sir=2C I wish you go through this offer t...      1
737  To take your mind off the Balkans for a second...      0
740                       Pls keep the updates coming!      0
660  CHRIST BETHEL HOSPITAL 11 RUE ABOBOTE,ABIDJAN ...      1
411  sbwhoeopFriday February 5 2010 7:11 AMHRe: Bra...      0


C:\Users\Salva\AppData\Local\Temp\ipykernel_1504\1710886297.py:9: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [6]:
import re

def clean_text(text):
    # Convert to string to avoid errors
    processed_feature = str(text)

    # Remove all special characters
    processed_feature = re.sub(r'\W', ' ', processed_feature)
    
    # Remove all single characters
    processed_feature = re.sub(r'\b\w\b', '', processed_feature)
    
    # Remove single characters from the start
    processed_feature = re.sub(r'^\s*[a-zA-Z]\s+', '', processed_feature)
    
    # Substitute multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature).strip()
    
    # Remove prefixed 'b'
    processed_feature = re.sub(r"^b\s+", '', processed_feature)
    
    #Remove numbers
    text = re.sub(r'\d+', '', text)
        
    # Convert to lowercase
    processed_feature = processed_feature.lower()
    
    return processed_feature


In [13]:
# Clean the training and validation sets
train_data_cleaned = train_data.copy()  # Create a copy to avoid modifying the original DataFrame
val_data_cleaned = val_data.copy()

# Apply the clean_text function to the 'text' column
train_data_cleaned[text_column] = train_data_cleaned[text_column].apply(clean_text)
val_data_cleaned[text_column] = val_data_cleaned[text_column].apply(clean_text)

# Display the cleaned data to check
print(train_data_cleaned.head())
print(val_data_cleaned.head())


                                                  text  label
29   regards mr nelson smith kindly reply me on my ...      1
535  have not been able to reach oscar this am we a...      0
695  huma abedin b6i checking with pat on the 50k w...      0
557     can have it announced here on monday can today      0
836  bank of africaagence san pedro14 bp 1210 san p...      1
                                                  text  label
521  dear sir 2c wish you go through this offer to ...      1
737  to take your mind off the balkans for second s...      0
740                        pls keep the updates coming      0
660  christ bethel hospital 11 rue abobote abidjan ...      1
411  sbwhoeopfriday february 2010 11 amhre bravo br...      0


## Now let's work on removing stopwords
Remove the stopwords.

In [7]:
# Define stop words set
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    # Tokenize the text into words
    words = text.split()
    
    # Remove stopwords
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Join the remaining words back into a string
    return ' '.join(filtered_words)

train_data_cleaned['text'] = train_data_cleaned['text'].apply(remove_stopwords)
val_data_cleaned['text'] = val_data_cleaned['text'].apply(remove_stopwords)

# Display the cleaned data
print(train_data_cleaned.head())


NameError: name 'train_data_cleaned' is not defined

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#nltk.download('wordnet')
#nltk.download('omw-1.4')

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization
def lemmatize_text(text):
    words = text.split()  # Tokenize the text into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return ' '.join(lemmatized_words)  # Join back into a single string

In [15]:


# Apply lemmatization to your DataFrame
train_data_cleaned['text'] = train_data_cleaned['text'].apply(lemmatize_text)
val_data_cleaned['text'] = val_data_cleaned['text'].apply(lemmatize_text)

# Display the cleaned and lemmatized data
print(train_data_cleaned.head())


                                                  text  label
29   regard mr nelson smith kindly reply private em...      1
535      able reach oscar supposed send pdb 11 receive      0
695  huma abedin b6i checking pat 50k work jack jak...      0
557                             announced monday today      0
836  bank africaagence san pedro14 bp 1210 san pedr...      1


In [10]:
# some things were not cleaned properly so here is another go at it. 


def enhanced_clean_text(text):
    # Step 1: Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Step 2: Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Step 3: Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Step 4: Fix boundary issues (e.g., missing spaces between lowercase and uppercase letters)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    
    # Step 5: Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Step 6: Remove prefixed 'b' if present
    text = re.sub(r"^b'", '', text)
    
    # Step 7: Convert to lowercase
    text = text.lower()
    
    return text

# Reapply the updated cleaning function
#train_data_cleaned['text'] = train_data_cleaned['text'].apply(enhanced_clean_text)
#val_data_cleaned['text'] = val_data_cleaned['text'].apply(enhanced_clean_text)

# Display the updated cleaned data
#print(train_data_cleaned.head())


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [14]:
# Separate spam and ham messages
spam_messages = train_data[train_data['label'] == 1]['text']
ham_messages = train_data[train_data['label'] == 0]['text']


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Function to get top N words from a set of messages
def get_top_n_words(messages, n=10):
    vectorizer = CountVectorizer()
    word_count_vector = vectorizer.fit_transform(messages)
    
    # Sum up word counts for each word
    word_count_sum = word_count_vector.sum(axis=0)
    
    # Create a dictionary mapping words to their frequency
    word_freq = [(word, word_count_sum[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    
    # Sort the dictionary by frequency and get top n words
    word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)
    
    # Return top n words
    return word_freq[:n]

# Get top 10 words for spam and ham messages
top_10_spam_words = get_top_n_words(spam_messages)
top_10_ham_words = get_top_n_words(ham_messages)

# Display the results
print("Top 10 words in spam messages:", top_10_spam_words)
print("Top 10 words in ham messages:", top_10_ham_words)


Top 10 words in spam messages: [('the', 5769), ('to', 4655), ('of', 4097), ('and', 3339), ('you', 2713), ('in', 2686), ('this', 2239), ('my', 1794), ('your', 1753), ('for', 1710)]
Top 10 words in ham messages: [('the', 1558), ('to', 933), ('and', 737), ('of', 694), ('in', 542), ('that', 367), ('for', 327), ('is', 327), ('on', 282), ('you', 255)]


## Extra features

In [11]:
# Load the original data
path_training = WindowsPath("C:/Users/Salva/Documents/AI Engeneering Ironhack/Week 4/lab-natural-language-processing/data/kg_train.csv")
data = pd.read_csv(path_training, encoding='latin-1')

# Define extra feature lists
money_simbol_list = r"|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = r"|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", 
                              "transaction", "win", "deposit", "password"])



# Display the updated dataset with the new features
print(data.head())

                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0


In [12]:
# Reduce the data (optional step, if necessary)
data = data.head(1000)  # Reduce to 1000 rows, for example
data['preprocessed_text'] = data['text'].apply(clean_text)
data['preprocessed_text'] = data['preprocessed_text'].apply(clean_html) 
data['preprocessed_text'] = data['preprocessed_text'].apply(remove_stopwords)
data['preprocessed_text'] = data['preprocessed_text'].apply(lemmatize_text)
data['preprocessed_text'] = data['preprocessed_text'].apply(enhanced_clean_text)


# Add the extra features to the original dataset
data['money_mark'] = data['preprocessed_text'].str.contains(money_simbol_list).astype(int)
data['suspicious_words'] = data['preprocessed_text'].str.contains(suspicious_words).astype(int)
data['text_len'] = data['preprocessed_text'].apply(len)

# Split the dataset into train and validation sets
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Display the shapes to confirm the split
print("Training set shape:", train_data.shape)
print("Validation set shape:", val_data.shape)


C:\Users\Salva\AppData\Local\Temp\ipykernel_28260\1710886297.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  comments = soup.findAll(text=lambda text: isinstance(text, Comment))


Training set shape: (800, 6)
Validation set shape: (200, 6)


In [13]:

# Apply lemmatization to your DataFrame
#train_data['preprocessed_text'] = train_data['preprocessed_text'].apply(lemmatize_text)
#val_data['preprocessed_text'] = val_data['preprocessed_text'].apply(lemmatize_text)

# Display the cleaned and lemmatized data
print(train_data.head())

                                                  text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   

                                     preprocessed_text  money_mark  \
29   regard mr nelson smith kindly reply private em...           1   
535         able reach oscar supposed send pdb receive           1   
695  huma abedin bi checking pat work jack jake res...           1   
557                             announced monday today           1   
836  bank africaagence san pedro bp san pedro cote ...           1   

     suspicious_words  text_len  
29                  0        79  
535                 0        42  
695                 0        79  
557                 0        22  
836               

# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [16]:

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from scipy.sparse import hstack

# Assuming your dataframe is 'data' with a 'preprocessed_text' column and new features
preprocessed_texts = data['preprocessed_text']

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the preprocessed text into a Bag of Words sparse matrix
X_bow = vectorizer.fit_transform(preprocessed_texts)

# Check the shape of the Bag of Words matrix
print("Shape of Bag of Words matrix:", X_bow.shape)


Shape of Bag of Words matrix: (1000, 21989)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'preprocessed_text' column contains the text you want to vectorize
texts = data['preprocessed_text']

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer and transform the entire dataset into TF-IDF representation
X_tfidf = tfidf_vectorizer.fit_transform(texts)

# Check the shape of the resulting TF-IDF matrix
print("Shape of the TF-IDF matrix:", X_tfidf.shape)

# Optional: Convert the sparse matrix to a dense matrix (memory-heavy)
# X_tfidf_dense = X_tfidf.toarray()
# print(X_tfidf_dense[:5])  # First 5 rows


Shape of the TF-IDF matrix: (1000, 21989)


## And the Train a Classifier?

In [19]:
from scipy.sparse import csr_matrix, hstack

# Step 1: Extract the new features as a DataFrame
new_features = data[['money_mark', 'suspicious_words', 'text_len']]

# Step 2: Convert the dense new features DataFrame into a sparse matrix
new_features_sparse = csr_matrix(new_features.values)

# Step 3: Combine the Bag of Words matrix with new features using hstack
X_combined = hstack([X_bow, new_features_sparse])

# Step 4: Check the shape of the combined feature matrix
print("Shape of the combined feature matrix:", X_combined.shape)


Shape of the combined feature matrix: (1000, 21992)


In [20]:
# Function to get top N words from the preprocessed text
def get_top_n_words(messages, n=10):
    vectorizer = CountVectorizer()
    word_count_vector = vectorizer.fit_transform(messages)
    
    # Sum up word counts for each word
    word_count_sum = word_count_vector.sum(axis=0)
    
    # Create a dictionary mapping words to their frequency
    word_freq = [(word, word_count_sum[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    
    # Sort the dictionary by frequency and get top n words
    word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)
    
    # Return top n words
    return word_freq[:n]

# Get top 10 words from the preprocessed text in spam and ham messages
spam_messages = data[data['label'] == 1]['preprocessed_text']
ham_messages = data[data['label'] == 0]['preprocessed_text']

top_10_spam_words = get_top_n_words(spam_messages)
top_10_ham_words = get_top_n_words(ham_messages)

# Display the top 10 words
print("Top 10 words in spam messages:", top_10_spam_words)
print("Top 10 words in ham messages:", top_10_ham_words)


Top 10 words in spam messages: [('br', 1103), ('money', 987), ('account', 898), ('bank', 801), ('fund', 780), ('transaction', 555), ('business', 514), ('country', 513), ('mr', 482), ('nbsp', 475)]
Top 10 words in ham messages: [('state', 169), ('pm', 127), ('would', 107), ('president', 99), ('time', 95), ('call', 94), ('mr', 91), ('obama', 84), ('percent', 81), ('secretary', 77)]


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'label' is your target column
y = data['label']

# Split the combined matrix into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Initialize the model (logistic regression as an example)
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on validation set
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.2f}")


Validation Accuracy: 0.97


c:\Users\Salva\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
# Load the test data
path_test = WindowsPath("C:/Users/Salva/Documents/AI Engeneering Ironhack/Week 4/lab-natural-language-processing/data/kg_test.csv")
test_data = pd.read_csv(path_test)

# Display the first few rows of the test data
print(test_data.head())


                                                text
0  usiness is for the fact that the deceased man ...
1  They are happy to adjust to the afternoon. I a...
2  Lael Brainard was confirmed 78-19 this afterno...
3  H <hrod17@clintonemail.com>Friday March 26 201...
4  n;"> Dear Good Friend,<br><br><br>I am happy t...


In [24]:
# Apply the same preprocessing steps to the test data
test_data['preprocessed_text'] = test_data['text'].apply(clean_text)
test_data['preprocessed_text'] = test_data['preprocessed_text'].apply(remove_stopwords)
test_data['preprocessed_text'] = test_data['preprocessed_text'].apply(lemmatize_text)

# Add new features to the test data
test_data['money_mark'] = test_data['preprocessed_text'].str.contains(money_simbol_list).astype(int)
test_data['suspicious_words'] = test_data['preprocessed_text'].str.contains(suspicious_words).astype(int)
test_data['text_len'] = test_data['preprocessed_text'].apply(len)


In [25]:
# Use the same CountVectorizer or TfidfVectorizer that was trained on the training data
X_test_bow = vectorizer.transform(test_data['preprocessed_text'])  # Apply vectorization on test data


In [26]:
from scipy.sparse import hstack
from scipy.sparse import csr_matrix

# Extract the new features from the test data
test_features = test_data[['money_mark', 'suspicious_words', 'text_len']]

# Convert the new features to a sparse matrix
test_features_sparse = csr_matrix(test_features.values)

# Combine BoW/TF-IDF matrix with new features
X_test_combined = hstack([X_test_bow, test_features_sparse])


In [27]:
# Use the trained model to predict the labels for the test data
y_test_pred = model.predict(X_test_combined)


In [28]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming the 'label' column exists in the test data
y_test_true = test_data['label']  # Replace with the actual label column in the test file

# Evaluate the accuracy of the model on the test data
test_accuracy = accuracy_score(y_test_true, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Print the classification report for more detailed metrics
print(classification_report(y_test_true, y_test_pred))


KeyError: 'label'

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code